Primer ilustruje koriscenje keras biblioteke u radu sa neuronskim mrezama. 
<br>
Zvanicni link: https://keras.io/
<br>
Bice prikazan primer kreiranja binarnog klasifikatora tumora. 

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.datasets import boston_housing
from sklearn import preprocessing

%matplotlib inline

In [3]:
np.random.seed(10)

In [4]:
# ucitavamo podatke 
(X_train, Y_train), (X_test, Y_test) = boston_housing.load_data(test_split=0.33)

In [6]:
# scaler = preprocessing.StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [7]:
N_train = X_train.shape[0]
N_test = X_test.shape[0]
number_of_features = X_train.shape[1]
N_train, N_test, number_of_features

(339, 167, 13)

In [8]:
input_size = number_of_features
output_size = 1

In [9]:
# pravimo sekvencijalni model 
model = Sequential()

In [10]:
# dodajemo slojeve modelu
model.add(Dense(input_shape=(input_size,), units=50))
model.add(Activation('relu'))
model.add(Dense(units=output_size))

In [11]:
# kompajliramo model
# bice koriscen Adam optimizator
# funkcija koja se optimizuje je srednjekvadratna greska
# mera koja se koristi za ocenu klasifikacije je tacnosti
model.compile(optimizer='adam', loss='mse', metrics=['acc'])

In [12]:
# ispisujemo informacije o modelu
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                700       
_________________________________________________________________
activation_1 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 751
Trainable params: 751
Non-trainable params: 0
_________________________________________________________________


In [13]:
# treniramo model
model.fit(X_train, Y_train, epochs=24, batch_size=32)

Epoch 1/24
339/339 [==============================] - 0s - loss: 11290.7989 - acc: 0.0000e+00     
Epoch 2/24
339/339 [==============================] - 0s - loss: 2256.5908 - acc: 0.0000e+00     
Epoch 3/24
339/339 [==============================] - 0s - loss: 539.5917 - acc: 0.0029     
Epoch 4/24
339/339 [==============================] - 0s - loss: 652.0809 - acc: 0.0000e+00     
Epoch 5/24
339/339 [==============================] - 0s - loss: 446.0022 - acc: 0.0029         
Epoch 6/24
339/339 [==============================] - 0s - loss: 362.3332 - acc: 0.0029     

In [14]:
evaluation = model.evaluate(X_test, Y_test)

 32/167 [====>.........................] - ETA: 0s

In [15]:
evaluation

[85.08993891184916, 0.0]

In [16]:
# na nivou modela se mogu procitati raznorazna svojstva
model.metrics_names

['loss', 'acc']

In [17]:
# nauceni koeficijenti modela se mogu dobiti get_weights metodom
# za svaki sloj postoji posebna matrica koeficijenata
weights = model.get_weights()
for i in range(0, len(weights)):
    print weights[i].shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-17-5a581ba5afe7>, line 5)

In [56]:
weights[3]

array([ 0.01869928], dtype=float32)

In [106]:
# nauceni model se moze koristiti u zadacima predikcije 
# npr. vrednosti koje izracunava model za prvih 10 ulaza test skupa
# model.predict(scaler.transform(X_test[0:10]))
model.predict(X_test[0:10])

array([[ -98.35038757],
       [ -81.66169739],
       [-167.7651825 ],
       [ -76.06240082],
       [ -74.79261017],
       [ -95.50502014],
       [ -95.26343536],
       [-100.30163574],
       [-166.53945923],
       [ -73.15464783]], dtype=float32)

In [107]:
Y_test[0:10]

array([ 14.1 ,  13.15,  21.32,   8.05,   7.6 ,   3.32,   1.73,   9.5 ,
        23.34,   3.13])

In [108]:
# na nivou keras biblioteke postoje omotaci koji naucene modele cine kompatibilnim sa
# scikit-learn bibliotekom
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn import model_selection
from sklearn import metrics

In [29]:
# primer ilustruje koriscenje unakrsne validacije 

In [88]:
def baseline_model():
    model = Sequential()
    model.add(Dense(50, input_dim= input_size, kernel_initializer='normal', activation='relu'))
    model.add(Dense(output_size, kernel_initializer='normal'))
    model.compile(loss='mse', optimizer='adam')
    return model

In [89]:
regressor = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)

In [90]:
# broj grupa unakrsne validacije se zadaje kroz cv parametar
model_selection.cross_val_score(regressor, X_test, Y_test, cv=10, scoring='neg_mean_squared_error').mean()

-19.555141652632802